In [1]:
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [2]:
# Load Data
data_dir = './data/'
class_names = ["apple", "bat", "circle", "clock", "cloud",
               "crown", "diamond", "donut", "fish",
               "hot_dog", "lightning", "mountain", "skull",
               "smiley_face", "square", "star", "sun", "t-shirt", "tree"]

In [3]:
X, y = [], []

for label, class_name in enumerate(class_names):
    file_path = os.path.join(data_dir, f"{class_name}.npy")
    data = np.load(file_path)  # shape: (30000, 784)
    X.append(data)
    y.append(np.full((data.shape[0],), label))

In [4]:

X = np.vstack(X)  # shape: (570000, 784)
y = np.hstack(y)  # shape: (570000,)

In [5]:
# ✅ Proper Normalization for image pixel data
X = X.astype('float32') / 255.0

In [6]:
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size=0.5, random_state=42, stratify=y_)

In [7]:
# ✅ Improved and Simplified Model
model = Sequential([
    Dense(512, activation='relu', input_shape=(784,), kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.2),
    Dense(19, activation='linear')  # logits
])

c:\Users\mehim\anaconda3\envs\ml_env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Compile
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [9]:

# ✅ Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5)
]

In [10]:

# ✅ Increased batch size for stability
model.fit(
    X_train, y_train,
    validation_data=(X_cv, y_cv),
    epochs=100,
    batch_size=512,
    callbacks=callbacks,
    verbose=2
)

Epoch 1/100
891/891 - 15s - 17ms/step - accuracy: 0.7600 - loss: 0.9505 - val_accuracy: 0.8579 - val_loss: 0.6196 - learning_rate: 1.0000e-03
Epoch 2/100
891/891 - 12s - 13ms/step - accuracy: 0.8455 - loss: 0.6689 - val_accuracy: 0.8806 - val_loss: 0.5426 - learning_rate: 1.0000e-03
Epoch 3/100
891/891 - 12s - 13ms/step - accuracy: 0.8635 - loss: 0.6033 - val_accuracy: 0.8892 - val_loss: 0.5103 - learning_rate: 1.0000e-03
Epoch 4/100
891/891 - 12s - 13ms/step - accuracy: 0.8736 - loss: 0.5671 - val_accuracy: 0.8961 - val_loss: 0.4875 - learning_rate: 1.0000e-03
Epoch 5/100
891/891 - 11s - 13ms/step - accuracy: 0.8801 - loss: 0.5430 - val_accuracy: 0.8981 - val_loss: 0.4788 - learning_rate: 1.0000e-03
Epoch 6/100
891/891 - 12s - 13ms/step - accuracy: 0.8845 - loss: 0.5298 - val_accuracy: 0.9026 - val_loss: 0.4647 - learning_rate: 1.0000e-03
Epoch 7/100
891/891 - 12s - 14ms/step - accuracy: 0.8868 - loss: 0.5191 - val_accuracy: 0.9049 - val_loss: 0.4550 - learning_rate: 1.0000e-03
Epoch 

In [11]:
# Evaluation
logits_cv = model.predict(X_cv)
y_cv_pred = tf.argmax(tf.nn.softmax(logits_cv, axis=1), axis=1).numpy()
print("Validation Accuracy:", accuracy_score(y_cv, y_cv_pred))

logits_test = model.predict(X_test)
y_test_pred = tf.argmax(tf.nn.softmax(logits_test, axis=1), axis=1).numpy()
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

1782/1782 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
Validation Accuracy: 0.9297017543859649
1782/1782 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Test Accuracy: 0.9312631578947368


In [ ]:
# Save model and normalization
model.save("doodle_model.keras")

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=doodle_model.